# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
# Import + Folders
from pathlib import Path
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest
from sklearn.linear_model import Ridge
from sklearn.metrics import precision_score, recall_score, f1_score, mean_absolute_error, mean_squared_error

In [ ]:
# ✅ EDIT THIS:
# Point this to the folder where you unzipped the Kaggle NAB dataset.
# Example:
# NAB_ROOT = Path(r"/Users/you/Downloads/nab")
NAB_ROOT = Path(r"PUT_YOUR_UNZIPPED_NAB_FOLDER_PATH_HERE")

assert NAB_ROOT.exists(), f"NAB_ROOT does not exist: {NAB_ROOT}"

# Project output folders (in your repo)
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
IMAGES_DIR = PROJECT_ROOT / "images"
for d in [DATA_DIR, ARTIFACTS_DIR, IMAGES_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("NAB_ROOT:", NAB_ROOT)
print("Outputs:", DATA_DIR, ARTIFACTS_DIR, IMAGES_DIR)

In [ ]:
# Discover a good AWS CloudWatch time series
# NAB commonly has: data/realAWSCloudwatch/*.csv
candidate_dir = None

# try common layouts (Kaggle zip may already have these paths)
for p in [
    NAB_ROOT / "data" / "realAWSCloudwatch",
    NAB_ROOT / "realAWSCloudwatch",
    NAB_ROOT / "data" / "realAWSCloudwatch".lower(),  # just in case
]:
    if p.exists():
        candidate_dir = p
        break

assert candidate_dir is not None, "Could not find realAWSCloudwatch folder under NAB_ROOT."

csv_files = sorted(candidate_dir.glob("*.csv"))
assert len(csv_files) > 0, f"No CSVs found in {candidate_dir}"

print(f"Found {len(csv_files)} AWS CloudWatch CSV files in:\n  {candidate_dir}\n")
print("Sample files:")
for f in csv_files[:10]:
    print(" -", f.name)

# Pick one "good" file automatically: prefer CPU utilization, else first file
preferred = [f for f in csv_files if "cpu_utilization" in f.name.lower()]
series_path = preferred[0] if preferred else csv_files[0]
print("\nSelected series:", series_path.name)

In [ ]:
# Load the CSV robustly (auto-detect timestamp/value)
def load_nab_csv(path: Path) -> pd.DataFrame:
    """
    NAB CSVs are typically two columns: timestamp,value
    but we auto-detect to be safe.
    """
    df = pd.read_csv(path)

    # If header exists and looks like timestamp/value
    cols = [c.lower() for c in df.columns]
    if "timestamp" in cols:
        ts_col = df.columns[cols.index("timestamp")]
    else:
        ts_col = df.columns[0]

    if "value" in cols:
        val_col = df.columns[cols.index("value")]
    else:
        # assume second column is the series value
        val_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]

    out = df[[ts_col, val_col]].copy()
    out.columns = ["timestamp", "value"]

    out["timestamp"] = pd.to_datetime(out["timestamp"], errors="coerce", utc=True)
    out = out.dropna(subset=["timestamp", "value"])
    out = out.sort_values("timestamp").reset_index(drop=True)

    # ensure numeric values
    out["value"] = pd.to_numeric(out["value"], errors="coerce")
    out = out.dropna(subset=["value"])

    out = out.set_index("timestamp")
    return out

ts = load_nab_csv(series_path)
print(ts.head())
print("\nRows:", len(ts))
print("Index tz:", ts.index.tz)

In [ ]:
# Load anomaly windows (ground truth) from NAB labels
def find_combined_windows_json(root: Path) -> Path:
    candidates = [
        root / "labels" / "combined_windows.json",
        root / "nab" / "labels" / "combined_windows.json",
        root / "combined_windows.json",
        root / "labels" / "combined_windows.json".lower(),
    ]
    for c in candidates:
        if c.exists():
            return c
    raise FileNotFoundError("Could not find labels/combined_windows.json under NAB_ROOT.")

labels_path = find_combined_windows_json(NAB_ROOT)
print("Found labels file:", labels_path)

with open(labels_path, "r") as f:
    combined_windows = json.load(f)

# Keys are like "realAWSCloudwatch/ec2_cpu_utilization_53ea38.csv"
# We need to match the relative path style used by NAB.
def infer_series_key(series_path: Path) -> str:
    name = series_path.name
    parent = series_path.parent.name  # expected: realAWSCloudwatch
    return f"{parent}/{name}"

series_key = infer_series_key(series_path)
print("Series key guess:", series_key)

assert series_key in combined_windows, (
    "This series was not found in combined_windows.json.\n"
    f"Tried key: {series_key}\n"
    "If this happens, print keys and choose the right one."
)

windows = combined_windows[series_key]
print("Number of labeled anomaly windows:", len(windows))
print("First 3 windows:", windows[:3])

In [ ]:
# Build y_true vector (1 if timestamp inside any window)
def build_labels(index_utc: pd.DatetimeIndex, windows) -> np.ndarray:
    y = np.zeros(len(index_utc), dtype=int)
    for w in windows:
        start = pd.to_datetime(w[0], utc=True)
        end = pd.to_datetime(w[1], utc=True)
        y |= ((index_utc >= start) & (index_utc <= end)).astype(int)
    return y

# Ensure series index is UTC
ts_utc = ts.copy()
if ts_utc.index.tz is None:
    ts_utc.index = ts_utc.index.tz_localize("UTC")
else:
    ts_utc.index = ts_utc.index.tz_convert("UTC")

y_true = build_labels(ts_utc.index, windows)

print("Total labeled anomaly points:", int(y_true.sum()))

In [ ]:
# Resample to regular frequency (helps both detection + forecasting)
# Infer approximate frequency
delta = ts_utc.index.to_series().diff().median()
print("Median time delta:", delta)

# Choose a resample rule based on median delta
# If it's around 5 minutes -> "5min", around 1 minute -> "1min", else fallback to median minutes
minutes = max(1, int(round(delta.total_seconds() / 60))) if pd.notna(delta) else 5
rule = f"{minutes}min"
print("Resample rule:", rule)

ts_r = ts_utc["value"].resample(rule).mean().interpolate("time")
y_r = pd.Series(y_true, index=ts_utc.index).resample(rule).max().fillna(0).astype(int).values

df = pd.DataFrame({"value": ts_r.values, "y_true": y_r}, index=ts_r.index)
df.head(), df["y_true"].sum()

In [ ]:
# Feature engineering for anomaly detection
def make_features(series: pd.Series, window=12) -> pd.DataFrame:
    X = pd.DataFrame({"value": series})
    X["diff"] = X["value"].diff()
    X["roll_mean"] = X["value"].rolling(window).mean()
    X["roll_std"] = X["value"].rolling(window).std()
    X = X.dropna()
    return X

X = make_features(df["value"], window=12)
y = df.loc[X.index, "y_true"].values

print("X shape:", X.shape)
print("y anomalies:", int(y.sum()))

In [ ]:
#  Baseline detector (rolling z-score residual)
resid = (X["value"] - X["roll_mean"]) / (X["roll_std"] + 1e-8)
Z_THRESHOLD = 3.0
pred_baseline = (np.abs(resid.values) > Z_THRESHOLD).astype(int)

baseline_metrics = {
    "precision": precision_score(y, pred_baseline, zero_division=0),
    "recall": recall_score(y, pred_baseline, zero_division=0),
    "f1": f1_score(y, pred_baseline, zero_division=0),
}
baseline_metrics

In [ ]:
# IsolationForest detector
iso = IsolationForest(
    n_estimators=300,
    contamination=0.03,
    random_state=42
)
iso.fit(X[["value", "diff", "roll_mean", "roll_std"]])

pred_iso = (iso.predict(X[["value", "diff", "roll_mean", "roll_std"]]) == -1).astype(int)

iso_metrics = {
    "precision": precision_score(y, pred_iso, zero_division=0),
    "recall": recall_score(y, pred_iso, zero_division=0),
    "f1": f1_score(y, pred_iso, zero_division=0),
}
iso_metrics

In [ ]:
# Plot detections (true anomalies vs predicted)
plot_df = pd.DataFrame({
    "value": X["value"],
    "true_anom": y,
    "baseline_anom": pred_baseline,
    "iso_anom": pred_iso
}, index=X.index)

fig, ax = plt.subplots(figsize=(12, 4))
plot_df["value"].plot(ax=ax)
ax.scatter(plot_df.index[plot_df["true_anom"] == 1], plot_df.loc[plot_df["true_anom"] == 1, "value"], marker="x", label="True anomaly")
ax.scatter(plot_df.index[plot_df["iso_anom"] == 1], plot_df.loc[plot_df["iso_anom"] == 1, "value"], label="IsolationForest")
ax.set_title(f"Anomaly Detection on {series_path.name}")
ax.set_xlabel("time")
ax.set_ylabel("value")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Save metrics + plot + the resampled series
comparison = pd.DataFrame([
    {"model": "baseline_zscore", **baseline_metrics},
    {"model": "isolation_forest", **iso_metrics},
]).sort_values("f1", ascending=False)

metrics_out = ARTIFACTS_DIR / "anomaly_metrics.json"
comparison.to_json(metrics_out, orient="records", indent=2)

data_out = DATA_DIR / f"{series_path.stem}_resampled.csv"
df.to_csv(data_out, index=True)

plot_out = IMAGES_DIR / "anomaly_plot.png"
fig, ax = plt.subplots(figsize=(12, 4))
plot_df["value"].plot(ax=ax)
ax.scatter(plot_df.index[plot_df["true_anom"] == 1], plot_df.loc[plot_df["true_anom"] == 1, "value"], marker="x", label="True anomaly")
ax.scatter(plot_df.index[plot_df["iso_anom"] == 1], plot_df.loc[plot_df["iso_anom"] == 1, "value"], label="IsolationForest")
ax.set_title(f"Anomaly Detection on {series_path.name}")
ax.legend()
plt.tight_layout()
plt.savefig(plot_out, dpi=150)
plt.show()

print("Saved:", metrics_out)
print("Saved:", data_out)
print("Saved:", plot_out)
comparison

In [ ]:
# Create lag features
def make_lagged_features(series: pd.Series, lags=24):
    df_lag = pd.DataFrame({"y": series})
    for i in range(1, lags + 1):
        df_lag[f"lag_{i}"] = df_lag["y"].shift(i)
    df_lag = df_lag.dropna()
    return df_lag

LAGS = 24  # adjust based on your resample rule (e.g., 24 steps)
lag_df = make_lagged_features(df["value"], lags=LAGS)

# Time-based split (no shuffling)
split = int(len(lag_df) * 0.8)
train = lag_df.iloc[:split]
test = lag_df.iloc[split:]

X_train, y_train = train.drop(columns=["y"]), train["y"]
X_test, y_test = test.drop(columns=["y"]), test["y"]

X_train.shape, X_test.shape

In [ ]:
# Train a simple forecaster (Ridge regression)
model = Ridge(alpha=1.0)
model.fit(X_train, y_train)

pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse = mean_squared_error(y_test, pred, squared=False)

print("Forecast MAE:", mae)
print("Forecast RMSE:", rmse)

In [ ]:
# Plot forecast vs actual
fig, ax = plt.subplots(figsize=(12, 4))
pd.Series(y_test.values, index=y_test.index).plot(ax=ax, label="Actual")
pd.Series(pred, index=y_test.index).plot(ax=ax, label="Forecast")
ax.set_title("Forecast vs Actual (Ridge + lag features)")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Save forecast artifacts
forecast_out = ARTIFACTS_DIR / "forecast_metrics.json"
with open(forecast_out, "w") as f:
    json.dump({"mae": float(mae), "rmse": float(rmse), "lags": LAGS, "model": "Ridge"}, f, indent=2)

forecast_plot_out = IMAGES_DIR / "forecast_plot.png"
fig, ax = plt.subplots(figsize=(12, 4))
pd.Series(y_test.values, index=y_test.index).plot(ax=ax, label="Actual")
pd.Series(pred, index=y_test.index).plot(ax=ax, label="Forecast")
ax.set_title("Forecast vs Actual (Ridge + lag features)")
ax.legend()
plt.tight_layout()
plt.savefig(forecast_plot_out, dpi=150)
plt.show()

print("Saved:", forecast_out)
print("Saved:", forecast_plot_out)